In [1]:

# ISOLTATION FOREST - FITNESS FUNCTION

# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import pandas as pd

# Load the data
df = pd.read_csv("../data/processed/CTU13_Normal_Traffic_20.csv")  # Replace with your file's path

# If you have labels and features separated, adjust accordingly. 
# Isolation Forest often works in an unsupervised manner without labels.
X = df  # You may select specific columns as features if needed

# Split the data into training and testing sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Initialize the Isolation Forest model
iso_forest = IsolationForest(random_state=42)

# Train the model on the training data
iso_forest.fit(X_train)

# Predict on the test data
# Values of -1 represent anomalies, 1 represents normal observations
predictions = iso_forest.predict(X_test)

# TODO: potem zrobic to samo juz bez splita - tylko dla bajery przetestowac na razie

In [2]:
print(predictions)

[ 1 -1  1 ...  1  1  1]


In [3]:

# AUTO ENCODER - ATTACKED MODEL

# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import pandas as pd

# Import necessary libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

df = pd.read_csv("../data/processed/CTU13_Normal_Traffic_20.csv") 
# Prepare the data (example without separating into training and testing sets)
# You might need to split and scale the data beforehand
X = df.values  # Assuming 'df' is your DataFrame

# Define the autoencoder structure
input_dim = X.shape[1]  # Number of features
encoding_dim = 32  # This can be adjusted according to your needs

# Input layer
input_layer = Input(shape=(input_dim,))
# Encoding layer
encoded = Dense(encoding_dim, activation='relu')(input_layer)
# Decoding layer
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Build the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X, X,  # Y is the same as X because we try to reconstruct the input
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_split=0.2)


KeyboardInterrupt: 

In [ ]:

import sys, os
sys.path.append(os.path.abspath('../'))

In [ ]:
from src.data.cic_preprocess import preprocess_traffic

df2 = preprocess_traffic('../data/interim/xflow.csv')

from sklearn.preprocessing import StandardScaler
import pandas as pd

X_anomalies = df2.values  # Przyjmujemy, że df2 to DataFrame z anomaliami

# Skalowanie danych (jeśli wcześniej skalowałeś dane treningowe)
scaler = StandardScaler().fit(X)  # Użyj tych samych parametrów skalowania co dla danych treningowych
X_anomalies_scaled = scaler.transform(X_anomalies)

# Użyj autoenkodera do rekonstrukcji danych zawierających anomalie
reconstructed = autoencoder.predict(X_anomalies_scaled)

# Oblicz błąd rekonstrukcji jako różnicę między danymi wejściowymi a rekonstrukcją
reconstruction_error = np.mean(np.power(X_anomalies_scaled - reconstructed, 2), axis=1)

# Możesz następnie ustalić próg, powyżej którego obserwacje będą uznawane za anomalie
# Prog można ustalić na podstawie błędu rekonstrukcji na danych treningowych (normalnych)
reconstruction_error_normal = np.mean(np.power(X - autoencoder.predict(X), 2), axis=1)
threshold = np.percentile(reconstruction_error_normal, 10)  # np. 95 percentyl błędu jako próg

# Klasyfikacja na podstawie progu
anomalies_predicted = reconstruction_error > threshold

# Teraz `anomalies_predicted` zawiera True dla przewidzianych anomalii i False dla normalnych obserwacji


334/334 [==============================] - 0s 240us/step


In [ ]:
sum(anomalies_predicted)

0

In [ ]:
df_mal = pd.read_csv("../data/processed/CTU13_attack_all_scenarios.csv")
df_mal.shape

(38898, 32)

In [ ]:
df_norm = pd.read_csv("../data/processed/CTU13_Normal_Traffic_80.csv")
df_norm.shape

(42651, 32)

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier

df_norm = pd.read_csv("../data/processed/CTU13_Normal_Traffic_80.csv")

#tutaj bedzie musiala byc wczytana kazda kombinacja wczytania wszysrkich plikow z botnet i konczacych sie na _processed.csv oprocz jednego - czyli jak bedzie tych plikow 10 to wytrenuj 10 catboostow
df_mal = ...

df_norm['label'] = 0
df_mal['label'] = 1

df_combined = pd.concat([df_norm, df_mal], ignore_index=True)

X = df_combined.drop('label', axis=1)
y = df_combined['label']

#tutaj daj jak sie da jakis random state dal reprodukowalnosci - w petli jakos wiele mdoeli
model = CatBoostClassifier(verbose=0)  # `verbose=0` wyłącza logowanie do konsoli dla czytelności
model.fit(X, y)

# i tutaj zapisanie tych wszystkich modeli catboost pod nazwami jak pliki tylko z dopiskiem _model, tzn. jak dla danego modelu brakuje plik3_processed.csv w treningu to ma sie nazywac plik3_model z odpowiednim rozszerzeniem

In [ ]:
df_to_predict = preprocess_traffic('../data/interim/xflow.csv')
predictions = model.predict(df_to_predict)

In [ ]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
sum(predictions)/len(predictions)

0.9992961287078934

In [ ]:
# jak sobie radzi z normalnymi

df_to_predict = pd.read_csv("../data/processed/CTU13_Normal_Traffic_20.csv")

In [ ]:
predictions = model.predict(df_to_predict)

In [ ]:
sum(predictions)/len(predictions)

0.004126418456344368

In [ ]:
# TODO: w wersji 100% bardzo skuteczne przewidywanie, sprawdzić przewidywanie po wyjęciu odpowiedniego sceanriusza:) - powinno być wystarczająco - należy zautomatyzować taki pipeline do testowania

# 1) końcowe modele i pipeline do szybkiego testowania wszystkiego []: tworzenie zbiorów[], tworzenie modeli koncowych[]
# 2) techniki perturbacji []
# 3) funkcja fitness;) - prawdop błąd śr. kw. []
# 4) spięcie wszystkiego w ten algorym i oglądanie parametrów/wyniki/interpretacja itp. []